# Generating Subsets of Wikidata

>Warning: 
**This notebook is under construction and it doesn't work**

## Purpose

>This notebook is used to create smaller subgraphs from a larger input Wikidata graph. Notebook users can provide a list of Wikidata classes (**QNodes**) to remove and preserve to create desired subsets of Wikidata. 


### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

UPDATE EXAMPLE INVOCATION


```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p wiki_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.wikibase-item.tsv.gz \
-p property_item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.property.wikibase-item.tsv.gz \
-p qual_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/qual.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no \
-p compute_pagerank no \
-p languages es,ru,zh-cn 
```

In [1]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import papermill as pm

import gzip

In [2]:
# Parameters

# Folder on local machine where to create the output and temporary folders
# output_path = "/Users/pedroszekely/Downloads/kypher"
# output_path = "/Users/markmann/Downloads/subset"
# output_path = "/nas/home/mbmann/subset"
output_path = "/nas/home/mbmann/subset2"

# The names of the output and temporary folders
output_folder = "output"
temp_folder = "temp.output"

# The location of input files
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4/"
# wiki_root_folder = "/Users/markmann/Google\ Drive/Shared\ drives/KGTK/datasets/wikidataos-v4-mm-2/"
wiki_root_folder = "/nas/home/mbmann/kgtk/datasets/wikidataos-v4-mm-2/"
# wiki_root_folder = "/nas/home/mbmann/KGTK-public-graphs2/wikidata-20201130/data/" #TODO: Create mount point

#TEST: For small dataset only 
metadata_folder = "/nas/home/mbmann/kgtk/datasets/wikidata-20200803-v5/data/"

claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"
qual_file = "qualifiers.tsv.gz"
property_datatypes_file = "metadata.property.datatypes.tsv.gz"
metadata_file = "metadata.types.tsv.gz" 
# isa_file = "derived.isa.tsv.gz" #Computed below
# p279star_file = "derived.P279star.tsv.gz" #Computed below

# Useful files Jupyter notebook
useful_files_notebook = "Wikidata Useful Files.ipynb"
# notebooks_folder = "/Users/markmann/Desktop/CKG/kgtk_subset/kgtk/examples/"
notebooks_folder = "/nas/home/mbmann/kgtk_subset/kgtk/examples/"

# Location of the cache database for kypher
# cache_path = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4"
cache_path = f'{output_path}/{output_folder}'

#Additional parameters
delete_database = "no"
compute_pagerank = "no"
languages = "en,"

### Needs fixing
# Whether to delete the cache database
if delete_database and delete_database.lower().strip() == 'yes':
    delete_database = True
else:
    delete_database = False

### Needs fixing
if compute_pagerank and compute_pagerank.lower().strip() == 'yes':
    compute_pagerank = True
else:
    compute_pagerank = False

if languages:
    languages = languages.split(',')

## Set up variables for files

In [3]:
#Environment variables
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

#Python variables
if cache_path:
    store = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    store = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

out = "{}/{}".format(output_path, output_folder)
temp = "{}/{}".format(output_path, temp_folder)

claims = wiki_root_folder + claims_file
labels = wiki_root_folder + label_file
aliases = wiki_root_folder + alias_file
descriptions = wiki_root_folder + description_file
items = wiki_root_folder + item_file
quals = wiki_root_folder + qual_file

#TEST
datatypes = metadata_folder + property_datatypes_file
metadata = metadata_folder + metadata_file

#LIVE
# datatypes = wiki_root_folder + property_datatypes_file
# metadata = wiki_root_folder + metadata_file

# shortcuts to commands
kgtk_path = "~/anaconda3/envs/kgtk-subset/bin/kgtk"
kgtk = f'time {kgtk_path} --debug'
kypher = f"{kgtk_path} query --debug --graph-cache " + store

Go to the output directory and create the subfolders for the output files and the temporary files

In [4]:
!cd $output_path
!mkdir {out}
!mkdir {temp}

mkdir: cannot create directory ‘/nas/home/mbmann/subset2/output’: File exists
mkdir: cannot create directory ‘/nas/home/mbmann/subset2/temp.output’: File exists


Clean up the output and temp folders before we start

In [5]:
# !rm {out}/*.tsv {out}/*.tsv.gz
# !rm {temp}/*.tsv {temp}/*.tsv.gz

if delete_database:
    !rm {out}/*.tsv {out}/*.tsv.gz
    !rm {temp}/*.tsv {temp}/*.tsv.gz

### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [6]:
!{kypher} -i {claims} \
--match '()-[]->()' \
--limit 10

[2021-02-14 11:58:19 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
id	node1	label	node2	rank	node2;wikidatatype
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"	normal	url
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	normal	url
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	normal	wikibase-item
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	normal	wikibase-property
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	normal	wikibase-property
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	normal	wikibase-property
P10-P1659-P51-86aca4c5-0	P10	P1659	P51	normal	wikibase-property
P10-P1855-Q7378-555592a4-0	P10	P1855	Q7378	normal	wikibase-item
P10-P31-Q18610173-85ef4d24-0	P10	P31	Q18610173	normal	wikibase-item
P1000-P1629-Q1241356-d5c10f50-0	P1000	P1629	Q1241356	normal	wikibase-item


## Compute derived.P279star and derived.isa files
Use claims file to compute derived files, used for initial processing. 
**NOTE:** We will later be creating smaller `derived.isa` and `derived.P279` files, when calling the useful-files notebook below. 

### Compute derived.P279star from claims

Create the `P31` file

In [7]:
!{kypher} -i {claims} -o {temp}/derived.P31.tsv.gz \
--match '(n1)-[l:P31]->(n2)' \
--return 'l, n1, l.label, n2' 

[2021-02-14 11:58:21 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P31']
---------------------------------------------
[2021-02-14 11:58:21 sqlstore]: CREATE INDEX on table graph_1 column label ...
[2021-02-14 11:58:42 sqlstore]: ANALYZE INDEX on table graph_1 column label ...


In [8]:
!zcat {temp}/derived.P31.tsv.gz | head | col

id	node1	label	node2
P10-P31-Q18610173-85ef4d24-0	P10	P31	Q18610173
P1000-P31-Q18608871-093affb5-0	P1000	P31	Q18608871
P1001-P31-Q15720608-deeedec9-0	P1001	P31	Q15720608

gzip: P1001-P31-Q22984026-8beb0cfe-0	P1001	P31	Q22984026
stdout: Broken pipe
P1001-P31-Q22997934-1e5b1a96-0	P1001	P31	Q22997934
P1001-P31-Q61719275-0ccc11a5-0	P1001	P31	Q61719275
P1001-P31-Q70564278-b92b04ba-0	P1001	P31	Q70564278
P1002-P31-Q22963600-b3a47587-0	P1002	P31	Q22963600
P1003-P31-Q19595382-152d2cdd-0	P1003	P31	Q19595382


Create the `P279` file

In [9]:
!{kypher} -i {claims} -o {temp}/derived.P279.tsv.gz \
--match '(n1)-[l:P279]->(n2)' \
--return 'l, n1, l.label, n2' 

[2021-02-14 11:59:27 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."id", graph_1_c1."node1", graph_1_c1."label", graph_1_c1."node2"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
  PARAS: ['P279']
---------------------------------------------


#### Create the file that contains all nodes reachable via P279 starting from a node2 in P31 or a node1 in P279
First compute the roots

In [10]:
!{kypher} -i {temp}/derived.P279.tsv.gz -o {temp}/P279.n1.tsv.gz \
--match '(n1)-[l]->()' \
--return 'n1 as id' 

[2021-02-14 11:59:43 sqlstore]: IMPORT graph directly into table graph_2 from /nas/home/mbmann/subset2/temp.output/derived.P279.tsv.gz ...
[2021-02-14 11:59:44 query]: SQL Translation:
---------------------------------------------
  SELECT graph_2_c1."node1" "_aLias.id"
     FROM graph_2 AS graph_2_c1
  PARAS: []
---------------------------------------------


In [11]:
!{kypher} -i {temp}/derived.P31.tsv.gz -o {temp}/P31.n2.tsv.gz \
--match '()-[l]->(n2)' \
--return 'n2 as id' 

[2021-02-14 11:59:48 sqlstore]: IMPORT graph directly into table graph_3 from /nas/home/mbmann/subset2/temp.output/derived.P31.tsv.gz ...
[2021-02-14 11:59:52 query]: SQL Translation:
---------------------------------------------
  SELECT graph_3_c1."node2" "_aLias.id"
     FROM graph_3 AS graph_3_c1
  PARAS: []
---------------------------------------------


In [12]:
!{kgtk} cat --mode NONE -i {temp}/P31.n2.tsv.gz {temp}/P279.n1.tsv.gz \
| gzip > {temp}/P279.roots.1.tsv.gz


real	0m17.552s
user	0m19.963s
sys	0m0.480s


In [13]:
!{kgtk} sort2 --mode NONE --column id -i {temp}/P279.roots.1.tsv.gz \
| gzip > {temp}/P279.roots.2.tsv.gz


real	0m3.885s
user	0m3.701s
sys	0m0.285s


We have lots of duplicates

In [14]:
!zcat < {temp}/P279.roots.2.tsv.gz | head

id
Q1
Q1
Q100000030
Q1000032
Q1000032
Q1000039
Q100004761
Q100004761
Q100004761

gzip: stdout: Broken pipe


In [15]:
!{kgtk} compact -i {temp}/P279.roots.2.tsv.gz --mode NONE \
--presorted \
--columns id \
> {temp}/P279.roots.tsv


real	0m20.386s
user	0m19.901s
sys	0m0.126s


Now we can invoke the reachable-nodes command

In [16]:
!{kgtk} reachable-nodes \
--rootfile {temp}/P279.roots.tsv \
--selflink \
-i {temp}/derived.P279.tsv.gz \
| gzip > {temp}/P279.reachable.tsv.gz


real	1m46.103s
user	2m10.976s
sys	0m5.141s


In [17]:
!zcat < {temp}/P279.reachable.tsv.gz | head | col

node1	label	node2
Q100000030	reachable	Q100000030
Q100000030	reachable	Q14748
Q100000030	reachable	Q14745
Q100000030	reachable	Q1357761
Q100000030	reachable	Q223557
Q100000030	reachable	Q4406616
Q100000030	reachable	Q488383
Q100000030	reachable	Q35120
Q100000030	reachable	Q2424752

gzip: stdout: Broken pipe


The reachable-nodes command produces edges labeled `reachable`, so we need one command to rename them.

In [18]:
!{kypher} -i {temp}/P279.reachable.tsv.gz -o {temp}/P279star.1.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "P279star" as label, n2 as node2' 

[2021-02-14 12:02:30 sqlstore]: IMPORT graph directly into table graph_4 from /nas/home/mbmann/subset2/temp.output/P279.reachable.tsv.gz ...
[2021-02-14 12:02:41 query]: SQL Translation:
---------------------------------------------
  SELECT graph_4_c1."node1", ? "_aLias.label", graph_4_c1."node2" "_aLias.node2"
     FROM graph_4 AS graph_4_c1
  PARAS: ['P279star']
---------------------------------------------


Now we can concatenate these files to produce the final output

In [19]:
!{kgtk} sort2 -i {temp}/P279star.1.tsv.gz -o {temp}/P279star.2.tsv.gz


real	0m26.870s
user	0m26.382s
sys	0m1.292s


Make sure there are no duplicates

In [20]:
!{kgtk} compact --presorted -i {temp}/P279star.2.tsv.gz -o {temp}/P279star.3.tsv.gz


real	3m18.539s
user	3m17.949s
sys	0m0.189s


Add ids

In [21]:
!{kgtk} add-id --id-style node1-label-node2-num -i {temp}/P279star.3.tsv.gz -o {temp}/derived.P279star.tsv.gz
!zcat {temp}/derived.P279star.tsv.gz | head


real	3m15.355s
user	3m12.590s
sys	0m1.870s
node1	label	node2	id
Q100000030	P279star	Q100000030	Q100000030-P279star-Q100000030-0000
Q100000030	P279star	Q1357761	Q100000030-P279star-Q1357761-0000
Q100000030	P279star	Q14745	Q100000030-P279star-Q14745-0000
Q100000030	P279star	Q14748	Q100000030-P279star-Q14748-0000
Q100000030	P279star	Q15401930	Q100000030-P279star-Q15401930-0000
Q100000030	P279star	Q15621286	Q100000030-P279star-Q15621286-0000
Q100000030	P279star	Q16686448	Q100000030-P279star-Q16686448-0000
Q100000030	P279star	Q17537576	Q100000030-P279star-Q17537576-0000
Q100000030	P279star	Q223557	Q100000030-P279star-Q223557-0000

gzip: stdout: Broken pipe


Make a path variablle

In [22]:
p279star = f'{temp}/derived.P279star.tsv.gz'

### Compute derived.isa from claims
The idea is that `(n1)-[:isa]->(n2)` when `(n1)-[:P31]->(n2)` or `(n1)-[:P279]->(n2)`

We do this by concatenating the files and renaming the relation

In [23]:
!{kgtk} cat -i {temp}/derived.P31.tsv.gz {temp}/derived.P279.tsv.gz \
    | gzip > {temp}/isa.1.tsv.gz


real	0m21.953s
user	0m29.213s
sys	0m0.493s


In [24]:
!{kypher} -i {temp}/isa.1.tsv.gz -o {temp}/derived.isa.tsv.gz \
--match '(n1)-[]->(n2)' \
--return 'n1, "isa" as label, n2'
!zcat {temp}/derived.isa.tsv.gz | head

[2021-02-14 12:11:25 sqlstore]: IMPORT graph directly into table graph_5 from /nas/home/mbmann/subset2/temp.output/isa.1.tsv.gz ...
[2021-02-14 12:11:30 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1", ? "_aLias.label", graph_5_c1."node2"
     FROM graph_5 AS graph_5_c1
  PARAS: ['isa']
---------------------------------------------
node1	label	node2
P10	isa	Q18610173
P1000	isa	Q18608871
P1001	isa	Q15720608
P1001	isa	Q22984026
P1001	isa	Q22997934
P1001	isa	Q61719275
P1001	isa	Q70564278
P1002	isa	Q22963600
P1003	isa	Q19595382

gzip: stdout: Broken pipe


Make a path variable

In [25]:
isa = f'{temp}/derived.isa.tsv.gz'

## Creating a list of all the items  to remove

**Add classes to remove, based on <u>classes themselves</u> here:** <br>
- **Example:** Let's remove the class (videotape recording, 'Q34508')
- **NOTE:** This will only remove items that have a P31/P279 relation with the class

In [26]:
classes_to_remove = ['Q34508', 'Q7378']
# classes_to_remove = ['Q13442814', 'Q523', 'Q318', 'Q7318358', 'Q7187', 'Q11173', 'Q8054'] #Parameter: Add classes manually here
classes = ', '.join([f'"{c}"' for c in classes_to_remove])
print('classes: ', classes)

!{kypher} -i {claims} -o {temp}/classes.remove.manual.tsv.gz \
--match '(instance)-[:P279]->(c)' \
--where 'c in [{classes}]' \
--return 'c, instance, "p279" as label'

!zcat {temp}/classes.remove.manual.tsv.gz |head

classes:  "Q34508", "Q7378"
[2021-02-14 12:11:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_1_c1."node2", graph_1_c1."node1", ? "_aLias.label"
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node2" IN (?, ?))
  PARAS: ['p279', 'P279', 'Q34508', 'Q7378']
---------------------------------------------
node2	node1	label
Q34508	Q1926658	p279
Q34508	Q20892182	p279
Q7378	Q223694	p279
Q34508	Q23058567	p279
Q34508	Q25324511	p279
Q34508	Q2916762	p279
Q7378	Q3629117	p279
Q34508	Q48774253	p279
Q7378	Q5036943	p279


**Add classes to remove, based on <u>instances</u> here:**
- **Example:** Let's remove classes that are part of instance (Fireball, 'Q5451712'), (Bush, 'Q1017471'), and (Italin Grape Alle, 'Q67772833')
- **NOTE:** The expected class to remove is (whisky, 'Q281'), (beer, 'Q44'), (beer brand, Q15075508), and (ale, 'Q208385')

Specify the intances to remove

In [27]:
# instances_to_remove = ['Q5451712', 'Q1017471', 'Q67772833'] #Parameter: Add instances manually here
# instances = ', '.join([f'"{instance}"' for instance in instances_to_remove])

For all instances, get their **(P279, subclass)** from `claims.tsv.gz`

In [28]:
# !{kypher} -i {claims} -o {temp}/classes.remove.p279.tsv.gz \
# --match '(instance)-[:P279]->(c)' \
# --where 'instance in [{instances}]' \

# !zcat {temp}/classes.remove.p279.tsv.gz |head

For all instances, get their **(P31, class)** from `claims.tsv.gz`

In [29]:
# !{kypher} -i {claims} -o {temp}/classes.remove.p31.tsv.gz \
# --match '(instance)-[:P31]->(c)' \
# --where 'instance in [{instances}]' \

# !zcat {temp}/classes.remove.p31.tsv.gz |head

Concatenate all `classes.remove.manual`, `classes.remove.p31`, and `classes.remove.p279` into one file

In [30]:
#Adding from all 2 sources
# !{kgtk} cat -i {temp}/classes.remove.manual.tsv.gz -i {temp}/classes.remove.p31.tsv.gz -i {temp}/classes.remove.p279.tsv.gz \
# -o {temp}/classes.remove.tsv.gz

#Adding from all just 1 sources
!{kgtk} cat -i {temp}/classes.remove.manual.tsv.gz \
-o {temp}/classes.remove.tsv.gz


real	0m1.539s
user	0m0.603s
sys	0m0.139s


Check and remove duplicate classes. 

In [31]:
!{kgtk} unique -i {temp}/classes.remove.tsv.gz \
-o {temp}/classes.remove2.tsv.gz

!zcat {temp}/classes.remove2.tsv.gz | head


real	0m1.012s
user	0m0.570s
sys	0m0.122s
node1	label	node2
Q34508	count	9
Q7378	count	4


### Compute the items to be removed via classes

First look at the classes we will remove

In [32]:
!zcat {temp}/classes.remove2.tsv.gz | head

node1	label	node2
Q34508	count	9
Q7378	count	4


1. Given all classes in `classes.remove2`, find all subclasses from `p279star`. <br>
2. Given all subclasses from `p279star`, find all subclass instances from `isa`
3. The resulting items to remove will be in `{temp}/items.remove.byclass.tsv.gz`

In [33]:
!{kypher} -i {temp}/classes.remove2.tsv.gz -i {p279star} -i {isa} \
--match 'isa: (item)-[:isa]->(subclass), P279star: (subclass)-[:P279star]->(c), class: (c)-[:count]->()' \
--return 'distinct item, "p31_p279star" as label, c as node2' \
-o {temp}/items.remove.byclass.tsv.gz

[2021-02-14 12:12:28 sqlstore]: IMPORT graph directly into table graph_6 from /nas/home/mbmann/subset2/temp.output/classes.remove2.tsv.gz ...
[2021-02-14 12:12:28 sqlstore]: IMPORT graph directly into table graph_7 from /nas/home/mbmann/subset2/temp.output/derived.P279star.tsv.gz ...
[2021-02-14 12:12:44 sqlstore]: IMPORT graph directly into table graph_8 from /nas/home/mbmann/subset2/temp.output/derived.isa.tsv.gz ...
[2021-02-14 12:12:47 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1", ? "_aLias.label", graph_6_c3."node1" "_aLias.node2"
     FROM graph_6 AS graph_6_c3, graph_7 AS graph_7_c2, graph_8 AS graph_8_c1
     WHERE graph_6_c3."label"=?
     AND graph_7_c2."label"=?
     AND graph_8_c1."label"=?
     AND graph_6_c3."node1"=graph_7_c2."node2"
     AND graph_7_c2."node1"=graph_8_c1."node2"
  PARAS: ['p31_p279star', 'count', 'P279star', 'isa']
---------------------------------------------
[2021-02-14 12:12:47 sqlstore]:

Check the result

In [34]:
!zcat {temp}/items.remove.byclass.tsv.gz | head
# !echo 'Johnnie Walker'
# !zgrep 'Q502268	' {temp}/items.remove.byclass.tsv.gz
# !echo 'Fireball'
# !zgrep 'Q5451712	' {temp}/items.remove.byclass.tsv.gz

node1	label	node2
Q33113219	p31_p279star	Q34508
Q39086719	p31_p279star	Q34508
Q58430369	p31_p279star	Q34508
Q61764640	p31_p279star	Q34508
Q61764914	p31_p279star	Q34508
Q69526230	p31_p279star	Q34508
Q69899824	p31_p279star	Q34508
Q99466084	p31_p279star	Q34508
Q99733532	p31_p279star	Q34508


### Compute the items to be removed via out degree

Specify the # of node out-degrees `k`, and identify items with out-degree less than `k`
- Ex: Find items that have out-degree `k` less than 2.

Compute out-degree for all QNodes in the `claims` file. Check the result.

In [35]:
# !{kypher} -i {claims} -o {temp}/metadata.out_degree.tsv.gz \
# --match '(n1)-[l]->()' \
# --where "upper(substr(n1,0)) >= 'Q'" \
# --return 'distinct n1 as node1, count(distinct l) as node2, "out_degree" as label' 

# !zcat {temp}/metadata.out_degree.tsv.gz | head

Create a list of items that have out_degree < `k`, along with any parent classses they belong to. <br>
Put the results into `items.remove.bydegree.tsv.gz`. <br>

In [36]:
# k = 2 #Parameter
# !{kypher} -i {temp}/metadata.out_degree.tsv.gz -i {isa} -i {p279star} \
# --match 'out: (item)-[:out_degree]->(n2), isa: (item)-[:isa]->(subclass), P279star: (subclass)-[:P279star]->(c)' \
# --where 'cast(n2, integer) <= {k}' \
# --return 'distinct item, "p31_p279star" as label, c as node2' \
# -o {temp}/items.remove.bydegree.tsv.gz \

# !zcat {temp}/items.remove.bydegree.tsv.gz | head

### Combine the items to remove by-class and by-outdegree
Concatenate all items from `items.remove.byclass` and `items.remove.bydegree`.
The resulting list of items to remove will be `items.remove`.

In [37]:
#If using class and outdegree sources
# !{kgtk} cat -i {temp}/items.remove.byclass.tsv.gz {temp}/items.remove.bydegree.tsv.gz \
# -o {temp}/items.remove.tsv.gz

#If just using class source
!{kgtk} cat -i {temp}/items.remove.byclass.tsv.gz \
-o {temp}/items.remove.tsv.gz


real	0m0.960s
user	0m0.438s
sys	0m0.087s


Deduplicate the concatenated file of items to remove. <br>
The resulting list of items to remove will be `items.remove2`.

In [38]:
!{kypher} -i {temp}/items.remove.tsv.gz -o {temp}/items.remove2.tsv.gz \
--match '(item)-[:p31_p279star]->(c)' \
--return 'distinct item, "p31_p279star" as label, c as node2'
!zcat {temp}/items.remove2.tsv.gz | head

[2021-02-14 12:13:20 sqlstore]: IMPORT graph directly into table graph_9 from /nas/home/mbmann/subset2/temp.output/items.remove.tsv.gz ...
[2021-02-14 12:13:20 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_9_c1."node1", ? "_aLias.label", graph_9_c1."node2" "_aLias.node2"
     FROM graph_9 AS graph_9_c1
     WHERE graph_9_c1."label"=?
  PARAS: ['p31_p279star', 'p31_p279star']
---------------------------------------------
[2021-02-14 12:13:20 sqlstore]: CREATE INDEX on table graph_9 column label ...
[2021-02-14 12:13:20 sqlstore]: ANALYZE INDEX on table graph_9 column label ...
node1	label	node2
Q33113219	p31_p279star	Q34508
Q39086719	p31_p279star	Q34508
Q58430369	p31_p279star	Q34508
Q61764640	p31_p279star	Q34508
Q61764914	p31_p279star	Q34508
Q69526230	p31_p279star	Q34508
Q69899824	p31_p279star	Q34508
Q99466084	p31_p279star	Q34508
Q99733532	p31_p279star	Q34508


### Validate the items we will remove
Check the `items.remove` file for classes added via different methods: 1) by-class, 2) by-instance, 3) by-outdegree

1) Check for class added manually, i.e. (videotape recording, 'Q34508')

In [39]:
# !echo 'videotape recording'
# !zgrep 'Q34508' {temp}/items.remove2.tsv.gz | head

2) Check for class added by-instance, i.e. (Fireball, 'Q5451712'), (whisky, 'Q281')

In [40]:
# !echo 'fireball'
# !zgrep 'Q5451712' {temp}/items.remove2.tsv.gz | head

# !echo 'whisky'
# !zgrep 'Q281' {temp}/items.remove2.tsv.gz | head

3) Check for class added by-outdegree, i.e. (??, 'Q100000030')

In [41]:
# !zgrep 'Q100000030' {temp}/items.remove2.tsv.gz | head

Collect all the classes of items we will remove, just as a sanity check

In [42]:
!{kypher} -i {temp}/items.remove2.tsv.gz \
--match '()-[]->(n2)' \
--return 'count(distinct n2)' \

!{kypher} -i {temp}/items.remove2.tsv.gz \
--match '()-[]->(n2)' \
--return 'distinct n2' \
--limit 10

[2021-02-14 12:13:23 sqlstore]: IMPORT graph directly into table graph_10 from /nas/home/mbmann/subset2/temp.output/items.remove2.tsv.gz ...
[2021-02-14 12:13:23 query]: SQL Translation:
---------------------------------------------
  SELECT count(DISTINCT graph_10_c1."node2")
     FROM graph_10 AS graph_10_c1
  PARAS: []
---------------------------------------------
count(DISTINCT graph_10_c1."node2")
2
[2021-02-14 12:13:25 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_10_c1."node2"
     FROM graph_10 AS graph_10_c1
     LIMIT ?
  PARAS: [10]
---------------------------------------------
node2
Q34508
Q7378


## [TODO] Create a list of all items to protect

## Create the reduced edges file

### Remove the items from the all.tsv and the label, alias and description files
We will be left with `reduced` files where the edges do not have the unwanted items. We have to remove them from the node1 and node2 positions, so we need to run the ifnotexists commands twice.

Before we start preview the files to see the column headings and check whether they look sorted.

In [43]:
!{kgtk} sort2 -i {temp}/items.remove2.tsv.gz -o {temp}/items.remove2.sorted.tsv.gz


real	0m1.029s
user	0m0.534s
sys	0m0.141s


In [44]:
!zcat < {temp}/items.remove2.sorted.tsv.gz | head | col
# !zgrep 'Q34508' {temp}/items.remove.sorted.tsv.gz -c #466

node1	label	node2
Q100000003	p31_p279star	Q34508
Q100000011	p31_p279star	Q34508
Q100000014	p31_p279star	Q34508
Q100000021	p31_p279star	Q34508
Q100000029	p31_p279star	Q34508
Q100000033	p31_p279star	Q34508
Q100000049	p31_p279star	Q34508
Q100328888	p31_p279star	Q34508
Q10267876	p31_p279star	Q34508


Remove from the full set of edges those edges that have a `node1` present in `items.remove.sorted.tsv`

In [45]:
!zcat {temp}/items.remove2.tsv.gz | head

node1	label	node2
Q33113219	p31_p279star	Q34508
Q39086719	p31_p279star	Q34508
Q58430369	p31_p279star	Q34508
Q61764640	p31_p279star	Q34508
Q61764914	p31_p279star	Q34508
Q69526230	p31_p279star	Q34508
Q69899824	p31_p279star	Q34508
Q99466084	p31_p279star	Q34508
Q99733532	p31_p279star	Q34508


In [46]:
!{kgtk} ifnotexists -i {claims} -o {temp}/item.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted 


real	4m51.754s
user	4m50.072s
sys	0m0.686s


From the remaining edges, remove those that have a `node2` present in `items.remove.sorted.tsv`

In [47]:
!{kgtk} sort2 -i {temp}/item.edges.reduced.tsv.gz -o {temp}/item.edges.reduced.sorted.tsv.gz \
--columns node2 label node1 id


real	2m21.681s
user	2m26.195s
sys	0m8.211s


In [48]:
!{kgtk} ifnotexists -i {temp}/item.edges.reduced.sorted.tsv.gz -o {temp}/item.edges.reduced.2.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node2 \
--filter-keys node1 \
--presorted 


real	5m2.719s
user	5m1.368s
sys	0m0.461s


Create a file with the labels

In [49]:
!{kgtk} ifnotexists -i {labels} -o {temp}/label.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	0m51.946s
user	0m45.047s
sys	0m0.234s


In [50]:
!{kgtk} sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.tsv.gz


real	0m12.793s
user	0m12.209s
sys	0m0.966s


Create a file with the aliases

In [51]:
!{kgtk} ifnotexists -i {aliases} -o {temp}/alias.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	0m18.436s
user	0m14.816s
sys	0m0.155s


Create a file with the descriptions

In [52]:
!{kgtk} ifnotexists -i {descriptions} -o {temp}/description.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove2.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted


real	0m27.398s
user	0m24.254s
sys	0m0.170s


### Produce the output files for claims, labels, aliases and descriptions

In [53]:
!{kgtk} sort2 -i {temp}/item.edges.reduced.2.tsv.gz -o {out}/claims.tsv.gz 


real	1m48.567s
user	1m48.229s
sys	0m7.856s


In [54]:
!{kgtk} sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.en.tsv.gz 


real	0m15.201s
user	0m13.092s
sys	0m1.024s


In [55]:
!{kgtk} sort2 -i {temp}/alias.edges.reduced.tsv.gz -o {out}/aliases.en.tsv.gz 


real	0m4.271s
user	0m3.998s
sys	0m0.358s


In [56]:
!{kgtk} sort2 -i {temp}/description.edges.reduced.tsv.gz -o {out}/descriptions.en.tsv.gz 


real	0m7.784s
user	0m7.383s
sys	0m0.747s


## Tests: Confirm items were removed from claims file
**NOTE:** We will check the items we removed do not exist in claims file

1) Confirm no instance of class added manually, i.e. (class: 'Q34508')<-(instance: 'Q100431477')

In [57]:
!zgrep 'Q100431477' {out}/claims.tsv.gz #PASS: No result

2) Confirm no target_instance of class added by source_instance, i.e. (source_instance, 'Q5451712')->(class:'Q281')<-(target_instance: Q1350656)

In [58]:
!zgrep 'Q1350656	' {out}/claims.tsv.gz #PASS: No result

Q1350656-P18-987dfd-4052c1a7-0	Q1350656	P18	"Georgiamoon.jpg"	normal	commonsMedia
Q1350656-P279-Q1007164-7e3ecba9-0	Q1350656	P279	Q1007164	normal	wikibase-item
Q1350656-P5456-8fa33b-317320ab-0	Q1350656	P5456	"corn-whiskey"	normal	external-id
Q1350656-P6366-be64d2-460bc253-0	Q1350656	P6366	"73756927"	normal	external-id
Q1350656-P646-1f74e6-81e29357-0	Q1350656	P646	"/m/01yr36"	normal	external-id


3) Confirm no instance with out-degree < 2, i.e. (instance: 'Q100000030')

In [59]:
!zgrep 'Q100000030	' {out}/claims.tsv.gz #PASS: No result

Q100000030-P18-a7b21b-2772e324-0	Q100000030	P18	"Brückentisch.jpg"	normal	commonsMedia
Q100000030-P279-Q14748-30394205-0	Q100000030	P279	Q14748	normal	wikibase-item


## Create the reduced qualifiers file
We do this by finding all the ids of the reduced edges file, and then selecting out from `qual.tsv`

We need to join by id, so we need to sort both files by id, node1, label, node2:

- `{quals}` 
- `{out}/claims.tsv.gz` 

In [60]:
!zcat < {quals} | head | column -t -s $'\t' 

id                                               node1                             label  node2                          node2;wikidatatype
P10-P1855-Q7378-555592a4-0-P10-8a982d-0          P10-P1855-Q7378-555592a4-0        P10    "Elephants Dream (2006).webm"  commonsMedia
P1000-P1896-f63a36-b84f3cd2-0-P1476-bf511b-0     P1000-P1896-f63a36-b84f3cd2-0     P1476  'FAI records'@en               monolingualtext
P1001-P1855-Q29868931-76b67d84-0-P1001-Q11736-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q11736                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q17269-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q17269                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q21208-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q21208                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q34800-0  P1001-P1855-Q29868931-76b67d84-0  P1001  Q34800                         wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q41079-0  

Run `ifexists` to select out the quals for the edges in `{out}/wikidataos.qual.tsv.gz`. Note that we use `node1` in the qualifier file, matching to `id` in the `wikidataos.all.tsv` file.

In [61]:
!$kgtk ifexists -i {quals} -o {out}/qualifiers.tsv.gz \
--filter-on {out}/claims.tsv.gz \
--input-keys node1 \
--filter-keys id \
--presorted


real	2m13.173s
user	2m9.064s
sys	0m0.286s


Look at the final output for qualifiers

In [62]:
!zcat < {out}/qualifiers.tsv.gz | head | col

id	node1	label	node2	node2;wikidatatype
P10-P1855-Q7378-555592a4-0-P10-8a982d-0 P10-P1855-Q7378-555592a4-0	P10	"Elephants Dream (2006).webm"	commonsMedia
P1000-P1896-f63a36-b84f3cd2-0-P1476-bf511b-0	P1000-P1896-f63a36-b84f3cd2-0	P1476	'FAI records'@en	monolingualtext
P1001-P1855-Q29868931-76b67d84-0-P1001-Q11736-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q11736	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q17269-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q17269	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q21208-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q21208	wikibase-item

gzip: P1001-P1855-Q29868931-76b67d84-0-P1001-Q34800-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q34800	wikibase-item
stdout: Broken pipe
P1001-P1855-Q29868931-76b67d84-0-P1001-Q41079-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q41079	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q42392-0 P1001-P1855-Q29868931-76b67d84-0	P1001	Q42392	wikibase-item
P1001-P1855-Q29868931-76b67d84-0-P1001-Q43684-0 P1001-P18

## Call partition and useful_files notebooks, to generate the file output

In [63]:
kgtk_scripts_path = "/nas/home/mbmann/kgtk_subset/kgtk"
os.environ["EXAMPLES_DIR"] = kgtk_scripts_path + "/examples"
os.environ["USECASE_DIR"] = kgtk_scripts_path + "/use-cases"
os.environ["TEMP"] = temp
os.environ["OUT"] = out
os.environ["DATATYPES"] = datatypes
os.environ["METADATA"] = metadata

In [64]:
os.environ["EXAMPLES_DIR"]

'/nas/home/mbmann/kgtk_subset/kgtk/examples'

In [65]:
!ls "$TEMP"

alias.edges.reduced.tsv.gz	  items.remove2.tsv.gz
classes.remove2.tsv.gz		  items.remove.byclass.tsv.gz
classes.remove.manual.tsv.gz	  items.remove.tsv.gz
classes.remove.tsv.gz		  label.edges.reduced.tsv.gz
derived.isa.tsv.gz		  P279.n1.tsv.gz
derived.P279star.tsv.gz		  P279.reachable.tsv.gz
derived.P279.tsv.gz		  P279.roots.1.tsv.gz
derived.P31.tsv.gz		  P279.roots.2.tsv.gz
description.edges.reduced.tsv.gz  P279.roots.tsv
isa.1.tsv.gz			  P279star.1.tsv.gz
item.edges.reduced.2.tsv.gz	  P279star.2.tsv.gz
item.edges.reduced.sorted.tsv.gz  P279star.3.tsv.gz
item.edges.reduced.tsv.gz	  P31.n2.tsv.gz
items.remove2.sorted.tsv.gz


In [66]:
!ls "$OUT"

aliases.en.tsv.gz	labels.en.tsv.gz   wikidata.sqlite3.db
claims.tsv.gz		labels.tsv.gz
descriptions.en.tsv.gz	qualifiers.tsv.gz


**Concatenate all output files together** <br>

**NOTE:** The `metadata.property.datatypes` and `metadata.types` are not currently generated by this notebook, and have been copied from `wikidata-20200803-v5/data`. <br>
**TODO:** We must confirm if these are source files, or computed. If they are computed, we should compute them in this notebook.

In [67]:
!{kgtk} cat \
-i {out}/aliases.en.tsv.gz \
-i {out}/descriptions.en.tsv.gz \
-i {out}/qualifiers.tsv.gz \
-i {out}/claims.tsv.gz \
-i {out}/labels.en.tsv.gz \
-i {datatypes} \
-i {metadata} \
-o {out}/all.tsv.gz


real	20m42.087s
user	20m33.929s
sys	0m1.026s


In [68]:
!ls {os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb"}

/nas/home/mbmann/kgtk_subset/kgtk/examples/partition-wikidata.ipynb


In [69]:
os.environ["OUT"]

'/nas/home/mbmann/subset2/output'

### Call the partition-wikidata notebook
`partition-wikidata` will take all intermediary computed outputs from `all.tsv.gz` and partition each wikidata entity (i.e. claims, aliases, labels, descriptions, qualifiers), into partitions. <br>

**NOTE:** This notebook also produces `claims.wikibase-item.tsv.gz` and outputs it to `wikidata_parts_path`

In [70]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = os.environ["OUT"] + "/all.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False
    )
)
;

Executing:   0%|          | 0/49 [00:00<?, ?cell/s]

''

## Call the useful-files notebook
`Wikidata Useful Files` will take intermediary output generated by `partition-wikidata` and produce the following statistics: `derived.P31.tsv.gz`, `derived.P279.tsv.gz`, `derived.isa.tsv.gz`, `derived.P279star.tsv.gz`, and `metadata.out_degree.tsv.gz`.

In [71]:
#NOTE: Don't pass in cache path, as one doesn't yet exist for useful_files
pm.execute_notebook(
    os.environ["USECASE_DIR"] + "/Wikidata Useful Files.ipynb",
    os.environ["TEMP"] + "/Wikidata Useful Files Out.ipynb",
    parameters=dict(
        output_path = os.environ["OUT"],
        output_folder = "useful_files",
        temp_folder = "temp.useful_files",
        wiki_root_folder = os.environ["OUT"] + "/parts/",
        languages = 'en',
        compute_pagerank = True,
        delete_database = False
    )
)
;

Executing:   0%|          | 0/103 [00:00<?, ?cell/s]

''

## Summary of results

In [72]:
!ls -lh {out}/*wikidataos.*

ls: cannot access /nas/home/mbmann/subset2/output/*wikidataos.*: No such file or directory


In [73]:
!zcat < {out}/wikidataos.all.tsv.gz | wc

/bin/bash: /nas/home/mbmann/subset2/output/wikidataos.all.tsv.gz: No such file or directory
      0       0       0


## Verification

The edges file must contain edges for properties, this is not the case on 2020-11-10`


In [74]:
!{kypher} -i {out}/claims.tsv.gz \
--match '(:P10)-[l]->(n2)' \
--limit 10

[2021-02-14 13:59:37 sqlstore]: IMPORT graph directly into table graph_11 from /nas/home/mbmann/subset2/output/claims.tsv.gz ...
[2021-02-14 14:00:17 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_11 AS graph_11_c1
     WHERE graph_11_c1."node1"=?
     LIMIT ?
  PARAS: ['P10', 10]
---------------------------------------------
[2021-02-14 14:00:17 sqlstore]: CREATE INDEX on table graph_11 column node1 ...
[2021-02-14 14:00:29 sqlstore]: ANALYZE INDEX on table graph_11 column node1 ...
id	node1	label	node2	rank	node2;wikidatatype
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"	normal	url
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"	normal	url
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508	normal	wikibase-item
P10-P1659-P1651-c4068028-0	P10	P1659	P1651	normal	wikibase-property
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18	normal	wikibase-property
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238	normal	wi

## concatenate files to get the `all` file

In [75]:
!{kgtk} cat -i {out}/claims.tsv.gz \
{out}/qualifiers.tsv.gz \
{out}/useful_files/metadata.pagerank.undirected.tsv.gz \
{out}/useful_files/metadata.pagerank.directed.tsv.gz \
{out}/useful_files/metadata.in_degree.sorted.tsv.gz \
{out}/useful_files/metadata.out_degree.sorted.tsv.gz \
-o {out}/wikidataos.all.tsv.gz


real	11m50.736s
user	11m49.087s
sys	0m0.672s


## concatenate files to get the `all for triples` file


In [76]:
!{kgtk} cat -i {out}/wikidataos.all.tsv.gz \
{out}/useful_files/derived.P31.tsv.gz \
{out}/useful_files/derived.P279.tsv.gz \
{out}/useful_files/derived.isa.tsv.gz \
{out}/useful_files/derived.P279star.tsv.gz \
-o {out}/wikidataos.all.for.triples.tsv.gz


real	16m41.457s
user	16m35.716s
sys	0m3.932s


## Filter out `novalue`, `somevalue` and `P9`

In [77]:
!{kgtk} filter -i {out}/wikidataos.all.for.triples.tsv.gz \
-o {out}/wikidataos.all.for.triples.filtered.tsv.gz \
-p ';;somevalue,novalue,P9' --invert


real	15m12.810s
user	15m10.494s
sys	0m0.773s


## Add ids for any edge with missing id

In [78]:
!{kgtk} add-id -i {out}/wikidataos.all.for.triples.filtered.tsv.gz \
-o {out}/wikidataos.all.for.triples.filtered.id.tsv.gz \
--id-style wikidata


real	15m44.195s
user	15m40.872s
sys	0m0.710s


## Sort by `id`

In [79]:
!{kgtk} sort2 -i {out}/wikidataos.all.for.triples.filtered.id.tsv.gz \
-o {out}/wikidataos.all.for.triples.filtered.id.sorted.tsv.gz \
-c id


real	4m13.014s
user	4m10.716s
sys	0m20.542s
